# ML2025 Homework 1 - Retrieval Augmented Generation with Agents

## Environment Setup

In this section, we install the necessary python packages and download model weights of the quantized version of LLaMA 3.1 8B. Also, download the dataset. Note that the model weight is around 8GB.

In [1]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

from pathlib import Path
if not Path('./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf').exists():
    !wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf
if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 148.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
ERROR: p

In [2]:
import torch
if not torch.cuda.is_available():
    raise Exception('You are not using the GPU runtime. Change it first or you will suffer from the super slow inference speed!')
else:
    print('You are good to go!')

You are good to go!


## Prepare the LLM and LLM utility function

By default, we will use the quantized version of LLaMA 3.1 8B. you can get full marks on this homework by using the provided LLM and LLM utility function. You can also try out different LLM models.

In the following code block, we will load the downloaded LLM model weights onto the GPU first.
Then, we implemented the generate_response() function so that you can get the generated response from the LLM model more easily.

You can ignore "llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized" warning.

In [3]:
from llama_cpp import Llama

# Load the model onto GPU
llama3 = Llama(
    "./Meta-Llama-3.1-8B-Instruct-Q8_0.gguf",
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


## Search Tool

The TA has implemented a search tool for you to search certain keywords using Google Search. You can use this tool to search for the relevant **web pages** for the given question. The search tool can be integrated in the following sections.

In [4]:
from typing import List
from googlesearch import search as _search
from bs4 import BeautifulSoup
from charset_normalizer import detect
import asyncio
from requests_html import AsyncHTMLSession
import urllib3
urllib3.disable_warnings()

async def worker(s:AsyncHTMLSession, url:str):
    try:
        header_response = await asyncio.wait_for(s.head(url, verify=False), timeout=10)
        if 'text/html' not in header_response.headers.get('Content-Type', ''):
            return None
        r = await asyncio.wait_for(s.get(url, verify=False), timeout=10)
        return r.text
    except:
        return None

async def get_htmls(urls):
    session = AsyncHTMLSession()
    tasks = (worker(session, url) for url in urls)
    return await asyncio.gather(*tasks)

async def search(keyword: str, n_results: int=3) -> List[str]:
    '''
    This function will search the keyword and return the text content in the first n_results web pages.
    Warning: You may suffer from HTTP 429 errors if you search too many times in a period of time. This is unavoidable and you should take your own risk if you want to try search more results at once.
    The rate limit is not explicitly announced by Google, hence there's not much we can do except for changing the IP or wait until Google unban you (we don't know how long the penalty will last either).
    '''
    keyword = keyword[:100]
    # First, search the keyword and get the results. Also, get 2 times more results in case some of them are invalid.
    results = list(_search(keyword, n_results * 2, lang="zh", unique=True))
    # Then, get the HTML from the results. Also, the helper function will filter out the non-HTML urls.
    results = await get_htmls(results)
    # Filter out the None values.
    results = [x for x in results if x is not None]
    # Parse the HTML.
    results = [BeautifulSoup(x, 'html.parser') for x in results]
    # Get the text from the HTML and remove the spaces. Also, filter out the non-utf-8 encoding.
    results = [''.join(x.get_text().split()) for x in results if detect(x.encode()).get('encoding') == 'utf-8']
    # Return the first n results.
    return results[:n_results]

## Test the LLM inference pipeline

In [5]:
# You can try out different questions here.
test_question='請問誰是 Taylor Swift？'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

泰勒絲（Taylor Swift）是一位美國歌手、詞曲作家和音樂製作人。她出生於1989年，來自田納西州。她的音乐风格从乡村摇滚发展到流行搖擺，並且她被誉为当代最成功的女艺人的之一。

泰勒絲早期在鄉郊小鎮演唱會時開始發展音樂事業，她推出了多張專輯，包括《Taylor Swift》、《Fearless》，以及後來更為知名的大熱作如 《1989》（2014年）、_reputation（）和 _Lover （）。她的歌曲經常探討愛情、友誼及自我成長等主題。

泰勒絲獲得了許多獎項，包括13座格萊美奖，並且是史上最快達到百萬銷量的女藝人之一。


## Agents

The TA has implemented the Agent class for you. You can use this class to create agents that can interact with the LLM model. The Agent class has the following attributes and methods:
- Attributes:
    - role_description: The role of the agent. For example, if you want this agent to be a history expert, you can set the role_description to "You are a history expert. You will only answer questions based on what really happened in the past. Do not generate any answer if you don't have reliable sources.".
    - task_description: The task of the agent. For example, if you want this agent to answer questions only in yes/no, you can set the task_description to "Please answer the following question in yes/no. Explanations are not needed."
    - llm: Just an indicator of the LLM model used by the agent.
- Method:
    - inference: This method takes a message as input and returns the generated response from the LLM model. The message will first be formatted into proper input for the LLM model. (This is where you can set some global instructions like "Please speak in a polite manner" or "Please provide a detailed explanation".) The generated response will be returned as the output.

In [6]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="bartowski/Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'bartowski/Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{self.task_description}\n{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            return ""

TODO 1: Design the role description and task description for each agent.

In [7]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是一位專業的問題分析師，擅長從複雜的敘述中找出真正需要解決的問題。你只會用繁體中文回答。",
    task_description="請從下列敘述中，萃取出最核心、需要解答的問題，並忽略與問題無關的背景或多餘資訊。只需輸出精簡明確的問題句。",
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是一位專業的關鍵字萃取專家，擅長從問題中找出最適合用來搜尋的關鍵字。你只會用繁體中文回答。",
    task_description="請從下列問題中，萃取出最適合用來搜尋的 2~5 個關鍵字或短語。只需輸出關鍵字，並以逗號分隔。",
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B，是用來回答問題的 AI。使用中文時只會使用繁體中文來回問題。",
    task_description="請回答以下問題：",
)

## RAG pipeline

TODO 2: Implement the RAG pipeline.

Please refer to the homework description slides for hints.

Also, there might be more heuristics (e.g. classifying the questions based on their lengths, determining if the question need a search or not, reconfirm the answer before returning it to the user......) that are not shown in the flow charts. You can use your creativity to come up with a better solution!

- Naive approach (simple baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive.png)

- Naive RAG approach (medium baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/naive_rag.png)

- RAG with agents (strong baseline)

    ![](https://www.csie.ntu.edu.tw/~ulin/rag_agent.png)

In [10]:
!pip install sentence-transformers chromadb langchain
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.4/441.4 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 99.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.3
    Uninstalling langsmith-0.2.3:
      Successfully uninstalled langsmith-0.2.3
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:


In [11]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# 1. 載入 embedding 模型
embedding_model = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")

<ipython-input-11-deba1e031b18>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
async def pipeline(question: str) -> str:
    # 1. 問題萃取
    core_question = question_extraction_agent.inference(question)
    
    # 2. 關鍵字萃取
    keywords = keyword_extraction_agent.inference(core_question)
    
    # 3. 搜尋網頁
    search_results = await search(keywords, n_results=5)  # 多抓幾筆，增加資訊多樣性
    
    # 4. 將每個搜尋結果切成小段（每500字一段）
    chunk_size = 500
    docs = []
    for doc in search_results:
        for i in range(0, len(doc), chunk_size):
            chunk = doc[i:i+chunk_size]
            if len(chunk) > 50:  # 過短的段落略過
                docs.append(chunk)
    
    # 5. 建立 Chroma 向量資料庫
    vector_db = Chroma.from_texts(texts=docs, embedding=embedding_model)
    
    # 6. 查詢最相關的段落（例如取前5段）
    top_k = 5
    relevant_docs_and_scores = vector_db.similarity_search_with_score(core_question, k=top_k)
    relevant_docs = [doc[0].page_content for doc in relevant_docs_and_scores]
    
    # 7. 對每段做摘要（可選，讓 context 更精簡）
    summaries = []
    for chunk in relevant_docs:
        summary = qa_agent.inference(f"請將以下資料摘要成100字重點：\n{chunk}")
        summaries.append(summary)
    context = "\n".join(summaries)
    
    # 8. 最終問答
    final_input = f"根據以下資料回答問題：\n{context}\n問題：{core_question}"
    answer = qa_agent.inference(final_input)
    return answer

In [13]:
result = await pipeline("請問2024年巴黎奧運的舉辦日期是什麼？請詳細說明。")
print(result)

根據資料，2024年巴黎奧運會的開幕式日期是7月26日，而閉門時間則是在8 月11 日。


## Answer the questions using your pipeline!

Since Colab has usage limit, you might encounter the disconnections. The following code will save your answer for each question. If you have mounted your Google Drive as instructed, you can just rerun the whole notebook to continue your process.

In [14]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "20250707"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"./{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = await pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'./{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)

1 根據資料，我們無法直接找到「虎山雄風飛揚」是哪間學校的校歌。然而，文中提到大同市第七中学啦拉操舞動青春熱情似火彰顯當代精神，這可能與你要找的是一樣東西
2 根據資料，NCC透過行政命令規定民眾境外郵購自用產品如無線鍑盤、滑鼠等，每案加收審查費750元。
3 根據資料，第一代 iPhone 的發表者是史蒂夫·乔布斯（Steve Jobs）。他在2007年1月9日推出iPhone時展現了其多功能性，並展示了一系列的特點，如音樂播放、視頻觀看、一鍵拨打電話和发送短信/電子郢等。
4 根據資料，部分美國大學需滿分90，而阿爾伯塔省加拿大研究所需要達到86。因此，如果你想申請進階英文免修，你應該至少要在托福網路測驗 TOEFL iBT 中取得 89 分以上的成績，以便能夠符合這些大學或研究生的入學要求。但是，建議參照各校官 網公告為準。
5 根據資料，觸地時間的目標是讓前腳掌落在身體正下方。
6 根據資料，卑南族的祖先發源地在巴拿馬。
7 根據資料，史蒂夫·乔布斯在里德学院学习期间的指導教授是罗伯特・弗理达兰（Robert Friedland）。
8 根據資料，詹姆斯·克拉 克麦克斯韦（James Clerk Maxwell）是發現電磁感應定律的人。
9 根據資料，距離國立臺灣史前文化博物館最近的臺鐵車站為康樂站在台南。
10 根據整數加法的性質，當 a 和 b 都是正数 時，其和為：a+b=|а+б|=20 + 30 = |50|  因此答案結果等於:| а |- б |=: **60**
11 根據資料，達拉斯獨行俠隊的Luka Doncic被交易至湖人（Los Angeles Lakers）球队。
12 根據 Associated Press 的報導，2024 年美國總統選舉結果如下：  *   民主党：喬拜登和凱米哈里斯获得总统候选人资格     *共和黨:唐納德川普再次成为総裁競爭對手
13 根據資料，Llama-3.2 系列模型中參數量最小的為 1B。
14 根據資料，學生每個学期可以停修不超過3門課程，但特殊情況可豁免限制。
15 根據資料，DeepSeek是一家中國公司，但沒有提到它的母 公司是誰。
16 根據資料，2024年NBA總冠軍隊伍是波士顿塞尔提克（Boston Celtics），但在你提供的新資訊中顯示了另一支球队奪得  Nba 總決賽。
17

In [16]:
import csv

STUDENT_ID = "20250707"
output_csv = f'./{STUDENT_ID}.csv'

# 讀取 public.txt 和 private.txt 的題目
questions = []
with open('./public.txt', 'r', encoding='utf-8') as f:
    questions += [l.strip().split(',')[0] for l in f.readlines()]
with open('./private.txt', 'r', encoding='utf-8') as f:
    questions += [l.strip().split(',')[0] for l in f.readlines()]

# 寫入 CSV
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Q', 'A'])  # 標題

    for idx, question in enumerate(questions, 1):
        ans_path = f'./{STUDENT_ID}_{idx}.txt'
        try:
            with open(ans_path, 'r', encoding='utf-8') as ans_f:
                answer = ans_f.readline().strip()
        except FileNotFoundError:
            answer = ''  # 若該題還沒跑完，答案留空
        writer.writerow([question, answer])

In [15]:
# Combine the results into one file.
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'./{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)

In [ ]:
import shutil
import os
from IPython.display import FileLink, display

STUDENT_ID = "20250707"

# 1. 指定要壓縮的檔案清單
files_to_zip = [f"{STUDENT_ID}.csv"]  # 先加總表
files_to_zip += [f"{STUDENT_ID}_{i}.txt" for i in range(1, 91)]  # 加入每題答案

# 2. 建立一個暫存資料夾，將所有檔案複製進去
tmp_dir = "tmp_zip"
os.makedirs(tmp_dir, exist_ok=True)
for file in files_to_zip:
    if os.path.exists(file):
        shutil.copy(file, tmp_dir)

# 3. 壓縮成 zip 檔
zip_name = f"{STUDENT_ID}_all_answers"
shutil.make_archive(zip_name, 'zip', tmp_dir)

# 4. 產生下載連結
display(FileLink(f"{zip_name}.zip"))

# 5. 清理暫存資料夾（可選）
shutil.rmtree(tmp_dir)